In [1]:
import random
import math

# Van Emde Boas

In [2]:
class Simple_Tabulation_Hash:
    def __init__(self, u, w):
        self.u = u
        self.w = w
        self.tab = [None for _ in range(w)]
        self.contador_elementos = 0
        self.contador_flags = 0
    
    def particionar(self, x):
        x = int(x)
        binario_x = bin(x).zfill(self.u).replace('b','')
        particoes = []
        if(self.u == 64 or self.u == 16 or self.u == 4):
            for _ in range(int(math.sqrt(self.u))):
                particoes.append(binario_x[-(int(math.sqrt(self.u))):])
                binario_x = binario_x[:-(int(math.sqrt(self.u)))]
        elif(self.u == 32):
            for _ in range(4):
                particoes.append(binario_x[-8:])
                binario_x = binario_x[:-8]

        elif(self.u == 8):
            for _ in range(2):
                particoes.append(binario_x[-4:])
                binario_x = binario_x[:-4]

        elif(self.u == 2):
             for _ in range(2):
                particoes.append(binario_x[-1:])
                binario_x = binario_x[:-1]
        else:
                print("Valor de u muito pequeno.")
        particoes = particoes[::-1]
        return particoes
        
    def tabela_aleatoria(self, x):
        random.seed(42)
        T = []
        particoes = self.particionar(x)
        for __ in range(len(particoes)):
            T_i = []
            for _ in range(2**len(particoes)):
                valor_aleatorio = random.randint(0,2**len(particoes))
                T_i.append(valor_aleatorio)
            T.append(T_i)
        return T
            
    def funcao_hash(self, x): #h(x) = T[1](x[1])^T[2](x[2])^...^T[8]x[8]
        particoes = self.particionar(x)
        particao_decimal = [int(particao, 2) for particao in particoes]
        T = self.tabela_aleatoria(x)
        T_j = []
        
        for i in range(len(particoes)):
            j = particao_decimal[i]
            T_j.append(T[i][j])

        h_x = T_j[0]^T_j[1]
        for i in range(2,len(particoes)):
            h_x = h_x^T_j[i]
            
        return h_x

    def limpar(self):
        tabela_antiga = self.tab.copy()
        self.tab = [None for _ in range(self.w)]
        print("LIMPAR:{}\n\n".format(self.contador_flags))
        self.remapear_valores(tabela_antiga)

    def remapear_valores(self, tabela_antiga):
        for pos, valor in enumerate(tabela_antiga):
            if(valor != None and valor != '#'):      # '#' indica que foi removido um elemento ali
                self.incluir(pos, valor, False) #Tem que modificar o self.tab ja com a inclusao do valor

    def table_doubling(self):
        tabela_antiga = self.tab.copy()
        m = len(self.tab)
        m *= 2
        self.tab = [None for _ in range(m)]
        self.remapear_valores(tabela_antiga)
        print("DOBRAR TAM:{}\n\n".format(len(self.tab)))

    def halving(self):
        eps=1
        if(len(self.tab)/2>=(eps+1)): #Aqui restringe a tabela a ter pelo menos tamanho 2.
            tabela_antiga = self.tab.copy()
            m = len(self.tab)
            m = int(m/2)
            self.tab = [None for _ in range(m)]
            self.remapear_valores(tabela_antiga)
            print("METADE TAM:{}\n\n".format(len(self.tab)))
            
    def incluir(self, x, veb, registrar_log=True):
        inserido = False
        i=0
        h_x = self.funcao_hash(x)
        while(not inserido):
            if(i<len(self.tab)):            #pra não percorrer a tabela infinitas vezes.
                pos = (h_x+i)%len(self.tab) #(h[x]+i)mod m
                
                if(self.tab[pos] == None or self.tab[pos] == '#'): # '#' indica que foi removido um elemento ali

                    if(veb in self.tab):
                        print("ELEMENTO JÁ INSERIDO")

                    elif(self.tab[pos] == '#'):
                        self.contador_flags-=1
                        
                    self.tab[pos] = veb
                    
                    inserido = True
                else:
                    i+=1
                    
            else:
                self.table_doubling() #se a tabela ja estiver cheia, table doubling remapeando valores.
                while(not inserido):
                    if(i<len(self.tab)):            
                        pos = (h_x+i)%len(self.tab) #(h[x]+i)mod m
                        if(self.tab[pos] == None or self.tab[pos] == '#'): # '#' indica que foi removido um elemento ali
                            if(self.tab[pos] == '#'):
                                self.contador_flags-=1  
                            self.tab[pos] = veb
                            inserido = True
                        i+=1

            self.contador_elementos+=1

        
    def buscar(self, x, veb, registrar_log=True):
        encontrou = False
        esta_na_tab = False
        h_x = self.funcao_hash(x)
        i=0
        print(len(self.tab))
        while(not encontrou):
            if(i<len(self.tab)):                 #Pra não percorrer a tabela infinitas vezes.
                pos = (h_x+i)%len(self.tab) #(h[x]+i)mod m
                if(self.tab[pos] == veb):          #Caso eu encontre o elemento, retorno sua posição.
                    return pos
                else:
                    i+=1
            else:
                encontrou = True
        
        return -1
            

    def remover(self, x, veb): #Implementar o halving (done)
        pos = self.buscar(x, veb, False)
        if(pos != -1):
            self.tab[pos] = '#'
            self.contador_elementos-=1
            self.contador_flags+=1
            if(self.contador_flags>=(len(self.tab))*(1/4)):
                self.limpar()

            if(self.contador_elementos<=int((1/4)*len(self.tab))):
                self.halving()

In [26]:
class VEB:
    def __init__(self, w):
        self.w = w
        self.w_2 = None
        self.u = 2**self.w
        self.min, self.max = None, None
        self.clusters = Simple_Tabulation_Hash(self.u, int(math.sqrt(self.u))) #[None for _ in range(int(math.sqrt(self.u)))]#
        self.resumo = None
        self.log = []
    
    def elemento(self, c, i):
         return (c << int(self.w_2,2)) or i #(c*math.floor(math.sqrt(self.u)))+i
 
    def posicao_chave(self, x):
        x = bin(x).zfill(int(self.w)).replace('b','')
        self.w_2 = x[:int(len(x)/2)]
        c = int(x,2) and ((1 << int(self.w_2,2)) - 1)
        i = int(x,2) >> int(self.w_2,2)
        #c = int(math.floor(x / math.sqrt(self.u)))
        #i = int(x % math.ceil(math.sqrt(self.u)))
        
        return c, i
    
    def atualizar_min_max(self, x):
        self.min = x
        self.max = x
    
    def inserir(self, x):
        c, i = self.posicao_chave(x)
        if(self.min == None):
            self.atualizar_min_max(x)
            self.log.append("INC:{}\n\n".format(x))
        else:
            if(x < self.min):
                aux = self.min
                self.min = x
                x = aux
            if(x > self.max):
                self.max = x
            if(self.clusters.tab[c] == None):
                #self.clusters.incluir(c, VEB(int((self.w)/2)))
                self.clusters.tab[c] = VEB((self.w)/2)
                self.log.append("INC:{}\n\n".format(x))
            if(self.resumo == None):
                self.resumo = VEB(int((self.w)/2))
            if(self.clusters.tab[c].min == None):
                self.resumo.inserir(c)
                self.clusters.tab[c].atualizar_min_max(i)
            else:
                self.clusters.tab[c].inserir(i)
                
    #V.clusters[c] que vai ser a tabela de dispersão, cada posição vai ser uma vEB. Exemplo: clusters[0] vai ter um 
    #ponteiro pra vEB que está na posição 0 da tabela de dispersão. Se c=0 (clusters[0]) então eu vou ver cluster[h(0)]
        
    def sucessor(self, x):  
        c, i = self.posicao_chave(x)
        if(x < self.min):
            return self.min
        else:
            max_i = None
            cluster = self.clusters.tab[c]
            if(cluster != None):
                max_i = cluster.max
            if(max_i != None and i < max_i):
                i_ = cluster.sucessor(i)
                self.log.append("SUC:{}\n{}\n\n".format(x, self.elemento(c, i_)))
                return self.elemento(c, i_)
            else:
                cluster_sucessor = None
                if(self.resumo != None):
                    cluster_sucessor = self.resumo.sucessor(c)
                if(cluster_sucessor == None):
                    return None
                else:
                    cluster_ = self.clusters.tab[cluster_sucessor]
                    i_ = 0
                    if(cluster_ != None):
                        i_ = cluster_.min
                    self.log.append("SUC:{}\n{}\n\n".format(x, self.elemento(cluster_sucessor, i_)))
                    return self.elemento(cluster_sucessor, i_)

    def predecessor(self, x):
        c, i = self.posicao_chave(x)
        if(x > self.max):
            return self.max
        else:
            min_i = None
            cluster = self.clusters.tab[c]
            if(cluster != None):
                min_i = cluster.min
            if(min_i != None and i > min_i):
                i_ = cluster.predecessor(i)
                self.log.append("PRE:{}\n{}\n\n".format(x, self.elemento(c, i_)))
                return self.elemento(c, i_)
            else:
                cluster_predecessor = None
                if(self.resumo != None):
                    cluster_predecessor = self.resumo.predecessor(c)
                if(cluster_predecessor == None):
                    if(self.min != None and x > self.min):
                        return self.min
                    return None
                else:
                    cluster_ = self.clusters.tab[cluster_predecessor]
                    i_ = 0
                    if(cluster_ != None):
                        i_ = cluster_.max
                    self.log.append("PRE:{}\n{}\n\n".format(x, self.elemento(cluster_predecessor, i_)))
                    return self.elemento(cluster_predecessor, i_)
                
    def remover(self, x):
        c,i = self.posicao_chave(x)
        if(x == self.min):
            c = self.resumo.min
            if(c == None):
                self.min = None
            i = self.clusters.tab[c].min
            self.min = self.elemento(c, i)
            x = self.min
        self.clusters.tab[c].remover(i)
        if(self.clusters.tab[c].min == None):
            self.resumo.remover(c)
        if(self.resumo.min == None):
            self.max = self.min
        else:
            c_ = self.resumo.max
            self.max = self.elemento(c_, self.clusters.tab[c_].max)
            
    def executar(self):
        arquivo = open('entrada.txt','r')
        saida = open('saida.txt','w')

        for line in arquivo:
            operacao, x = line.split(':')
            if(operacao == 'INC'):
                self.inserir(int(x))
                print(self.clusters.tab)

            elif(operacao == 'REM'):
                self.remover(int(x))
                print(self.clusters.tab)

            elif(operacao == 'SUC'):
                self.sucessor(int(x))
                
            elif(operacao == 'PRE'):
                self.predecessor(int(x))

        saida.writelines(self.log)

        arquivo.close()
        saida.close()


In [27]:
veb = VEB(16)
veb.u

65536

In [28]:
int(math.floor(10 / math.sqrt(4)))

5

In [29]:
veb.inserir(2)

In [30]:
veb.clusters

In [31]:
veb.min, veb.max

(2, 2)

In [32]:
veb.predecessor(2)

In [33]:
veb.min

2

In [34]:
veb.inserir(3)

In [35]:
#veb.inserir(2)

In [36]:
veb.predecessor(3)

2

In [37]:
veb.min, veb.max

(2, 3)

In [38]:
veb.inserir(12)

In [39]:
veb.clusters.tab

 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [40]:
veb.clusters.tab[0].clusters.tab, veb.clusters.tab[0].min, veb.clusters.tab[0].max

([<__main__.VEB at 0x222ceb21d30>,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 3,
 12)

In [41]:
veb.clusters.tab[0].clusters.tab[0].clusters.tab, veb.clusters.tab[0].clusters.tab[0].min, veb.clusters.tab[0].clusters.tab[0].max

([None, None, None, None], 12, 12)

In [42]:
veb.predecessor(12), veb.sucessor(3)

(3, 12)